In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
feature = pd.read_csv('../본실험3 - 변수선택용 proact 데이터/imputed features_final.csv') 
optimal_target = pd.read_csv('../본실험3 - 변수선택용 proact 데이터/Optimal_target_with_92_Q3_2점.csv') 
real_target = pd.read_csv('../본실험3 - 변수선택용 proact 데이터/Real_target_full_time.csv') 

In [4]:
opt_data = pd.merge(feature, optimal_target, on = 'SubjectID')
opt_data = opt_data.drop(columns=['SubjectID','Unnamed: 0_x', 'Unnamed: 0_y'])
opt_data = opt_data[['Age', 'Gender', 'onset_site', 'onset_delta',
       'diag_minus_onset', 'slope_alsfrs_tot',
       'fvc_slope', 'fvc_mean',  'mean_Q10_Respiratory',
       'mean_Bulbar', 'mean_motor', 'slope_Q1_Speech', 'slope_Q2_Salivation',
       'slope_Q3_Swallowing', 'slope_Q4_Handwriting', 'slope_Q5_Cutting',
       'slope_Q6_Dressing_and_Hygiene', 'slope_Q7_Turning_in_Bed',
       'slope_Q8_Walking', 'slope_Q9_Climbing_Stairs', 'slope_Q10_Respiratory', 
       'time_opt', 'status_opt']]

In [5]:
y = opt_data[['status_opt']]

In [6]:
from sklearn.model_selection import train_test_split

df_train, df_test, y_train, y_test = train_test_split(opt_data, y, train_size=0.8, test_size=0.2, random_state=11)

In [7]:
pip install lifelines

  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp37-none-any.whl size=4040 sha256=74a72cefbed0cac39c8a63b32fdc4502f2efc6321e09042623ee13a031ac4a91
  Stored in directory: C:\Users\pc\AppData\Local\pip\Cache\wheels\dc\68\dc\91321c55fba449755524481854f5be70d41912b8f886f908bb
  Created wheel for autograd: filename=autograd-1.3-cp37-none-any.whl size=47996 sha256=caa17a2bc968998a2e5188621cfb675e4f917c31e53fe932dae81bcce677eddb
  Stored in directory: C:\Users\pc\AppData\Local\pip\Cache\wheels\42\62\66\1121afe23ff96af4e452e0d15e68761e3f605952ee075ca99f
Successfully built autograd-gamma autograd
Note: you may need to restart the kernel to use updated packages.


In [102]:
initial_train = df_train.copy()
initial_feature = ['Age', 'Gender', 'onset_site', 'onset_delta',
       'diag_minus_onset', 'slope_alsfrs_tot',
       'fvc_slope', 'fvc_mean',  'mean_Q10_Respiratory',
       'mean_Bulbar', 'mean_motor', 'slope_Q1_Speech', 'slope_Q2_Salivation',
       'slope_Q3_Swallowing', 'slope_Q4_Handwriting', 'slope_Q5_Cutting',
       'slope_Q6_Dressing_and_Hygiene', 'slope_Q7_Turning_in_Bed',
       'slope_Q8_Walking', 'slope_Q9_Climbing_Stairs', 'slope_Q10_Respiratory']
SL = 0.05

In [103]:
from lifelines import CoxPHFitter

In [104]:
def cph_backward_elimination(initial_train, initial_feature, SL):
    
    train = initial_train.copy()
    feature_list = initial_feature.copy()
    
    for i in range(len(feature_list)):
        cph = CoxPHFitter(penalizer=0.01)
        cph.fit(train, duration_col='time_opt', event_col='status_opt')
        summary = cph.summary
        
        max = summary.reset_index().sort_values(by='p', ascending = False).iloc[0,:]
        max_covariate = max['covariate']
        max_p = max['p']

        if max_p > SL:
            feature_list.remove(max_covariate)
            train = train.drop([max_covariate], axis =1)
        else:
            break
    
    return train, feature_list

cph_selected_train, cph_selected_feature = cph_backward_elimination(initial_train, initial_feature, SL)

In [106]:
cph_selected_feature

['Age',
 'onset_site',
 'onset_delta',
 'slope_alsfrs_tot',
 'fvc_mean',
 'mean_Bulbar',
 'mean_motor',
 'slope_Q3_Swallowing',
 'slope_Q4_Handwriting']

In [107]:
cph.summary

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
covariate,,,,,,,,,,
Age,0.019854,1.020053,0.002760,0.014445,0.025263,1.014550,1.025585,7.194111,6.286886e-13,40.532720
onset_site,0.279860,1.322944,0.096069,0.091568,0.468151,1.095891,1.597039,2.913114,3.578435e-03,8.126455
onset_delta,0.000993,1.000993,0.000119,0.000759,0.001227,1.000759,1.001227,8.323433,8.545186e-17,53.377666
slope_alsfrs_tot,-2.798625,0.060894,0.506405,-3.791160,-1.806090,0.022569,0.164295,-5.526461,3.267550e-08,24.867215
fvc_mean,-0.006885,0.993138,0.002150,-0.011100,-0.002671,0.988961,0.997333,-3.202010,1.364721e-03,9.517178
mean_Bulbar,-0.743460,0.475466,0.030467,-0.803174,-0.683745,0.447905,0.504723,-24.402021,1.627751e-131,434.469701
mean_motor,-0.056815,0.944769,0.007394,-0.071307,-0.042323,0.931176,0.958560,-7.684104,1.540718e-14,45.883391
slope_Q3_Swallowing,-10.252306,0.000035,1.763194,-13.708103,-6.796509,0.000001,0.001118,-5.814620,6.077163e-09,27.293955
slope_Q4_Handwriting,5.565987,261.383062,2.107378,1.435602,9.696372,4.202176,16258.507294,2.641191,8.261516e-03,6.919378


In [118]:
from lifelines.utils import concordance_index

cph = CoxPHFitter(penalizer=0.01)
cph.fit(df_train, 'time_opt', event_col='status_opt')
concordance_index(df_test['time_opt'], -cph.predict_partial_hazard(df_test), df_test['status_opt'])

0.8396708549717625

In [119]:
from lifelines.utils import concordance_index

cph_selected_test = df_test[cph_selected_feature + ['time_opt','status_opt']]
cph = CoxPHFitter(penalizer=0.01)
cph.fit(cph_selected_train, 'time_opt', event_col='status_opt')
concordance_index(cph_selected_test['time_opt'], -cph.predict_partial_hazard(cph_selected_test), cph_selected_test['status_opt'])

0.839061709530545

In [112]:
from lifelines import WeibullAFTFitter

In [124]:
def aft_backward_elimination(initial_train, initial_feature, SL):
    
    train = initial_train.copy()
    feature_list = initial_feature.copy()
    
    for i in range(len(feature_list)):
        aft = WeibullAFTFitter()
        aft.fit(train, duration_col='time_opt', event_col='status_opt')
        summary = aft.summary
        
        max = summary.reset_index()[summary.reset_index()['covariate'] != 'Intercept'].sort_values(by='p', ascending = False).iloc[0,:]
        max_covariate = max['covariate']
        max_p = max['p']

        if max_p > SL:
            feature_list.remove(max_covariate)
            train = train.drop([max_covariate], axis =1)
        else:
            break
    
    return train, feature_list

aft_selected_train, aft_selected_feature = aft_backward_elimination(initial_train, initial_feature, SL)

In [125]:
aft_selected_feature

['Age',
 'onset_site',
 'onset_delta',
 'diag_minus_onset',
 'slope_alsfrs_tot',
 'fvc_mean',
 'mean_Bulbar',
 'mean_motor',
 'slope_Q3_Swallowing',
 'slope_Q4_Handwriting']

In [126]:
aft.summary

coef   exp(coef)  se(coef)  coef lower 95%  \
param   covariate                                                              
lambda_ Age                  -0.012938    0.987145  0.001742       -0.016351   
        diag_minus_onset     -0.000340    0.999660  0.000134       -0.000602   
        fvc_mean              0.004419    1.004429  0.001370        0.001735   
        mean_Bulbar           0.468420    1.597468  0.020376        0.428484   
        mean_motor            0.038309    1.039052  0.004877        0.028750   
        onset_delta          -0.000850    0.999150  0.000106       -0.001057   
        onset_site           -0.172941    0.841187  0.060930       -0.292362   
        slope_Q3_Swallowing   6.260001  523.219488  1.133556        4.038273   
        slope_Q4_Handwriting -3.542602    0.028938  1.355222       -6.198788   
        slope_alsfrs_tot      1.783521    5.950772  0.325624        1.145309   
        Intercept             2.667002   14.396748  0.188143        2.298250   
rho_    Intercept             0.477454    1.611966  0.023638        0.431125   

                              coef upper 95%  exp(coef) lower 95%  \
param   covariate                                                   
lambda_ Age                        -0.009525             0.983782   
        diag_minus_onset           -0.000078             0.999398   
        fvc_mean                    0.007104             1.001736   
        mean_Bulbar                 0.508356             1.534928   
        mean_motor                  0.047869             1.029167   
        onset_delta                -0.000643             0.998943   
        onset_site                 -0.053520             0.746499   
        slope_Q3_Swallowing         8.481729            56.728289   
        slope_Q4_Handwriting       -0.886416             0.002032   
        slope_alsfrs_tot            2.421733             3.143414   
        Intercept                   3.035755             9.956739   
rho_    Intercept                   0.523784             1.538987   

                              exp(coef) upper 95%          z              p  \
param   covariate                                                             
lambda_ Age                              0.990521  -7.429040   1.093890e-13   
        diag_minus_onset                 0.999922  -2.543911   1.096189e-02   
        fvc_mean                         1.007129   3.226478   1.253240e-03   
        mean_Bulbar                      1.662555  22.988901  6.019405e-117   
        mean_motor                       1.049033   7.854456   4.015111e-15   
        onset_delta                      0.999357  -8.054993   7.948333e-16   
        onset_site                       0.947887  -2.838348   4.534765e-03   
        slope_Q3_Swallowing           4825.786857   5.522448   3.343096e-08   
        slope_Q4_Handwriting             0.412130  -2.614038   8.947898e-03   
        slope_alsfrs_tot                11.265361   5.477238   4.320161e-08   
        Intercept                       20.816689  14.175432   1.300402e-45   
rho_    Intercept                        1.688405  20.198531   1.008524e-90   

                                -log2(p)  
param   covariate                         
lambda_ Age                    43.055597  
        diag_minus_onset        6.511360  
        fvc_mean                9.640121  
        mean_Bulbar           386.075966  
        mean_motor             47.823481  
        onset_delta            50.160197  
        onset_site              7.784756  
        slope_Q3_Swallowing    24.834240  
        slope_Q4_Handwriting    6.804236  
        slope_alsfrs_tot       24.464340  
        Intercept             149.107806  
rho_    Intercept             298.961284

In [122]:
from lifelines.utils import concordance_index

aft = WeibullAFTFitter()
aft.fit(df_train, 'time_opt', event_col='status_opt')
concordance_index(df_test['time_opt'], aft.predict_median(df_test), df_test['status_opt'])

0.8392165770156004

In [123]:
from lifelines.utils import concordance_index

aft_selected_test = df_test[aft_selected_feature + ['time_opt','status_opt']]
aft = WeibullAFTFitter()
aft.fit(aft_selected_train, 'time_opt', event_col='status_opt')
concordance_index(aft_selected_test['time_opt'], aft.predict_median(aft_selected_test), aft_selected_test['status_opt'])

0.8411885563253043

In [132]:
feature[(feature['onset_site']!= 0) & (feature['onset_site']!= 1)]

,Unnamed: 0,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,...,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
2918,2918,657543.0,46.0,1.0,-291.0,-770.0,-0.311379,479.0,-0.103619,82.5,...,0.0,0.0,-0.013158,-0.013158,0.026316,0.0,0.0,0.0,0.0,-0.039474
